# Import des bibliothèques nécessaires

In [1]:
import pandas as pd

# Récupération des données

In [2]:
url = 'https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv'
    
chipo = pd.read_csv(url, sep = '\t')

# 0. Regarde à  quoi ressemblent les données

1. Affiche quelques données
2. Regarde les types des données avec l'attribut .dtypes

In [3]:
chipo.head(10)

,order_id,quantity,item_name,choice_description,item_price
0,1,1,Chips and Fresh Tomato Salsa,NaN,$2.39
1,1,1,Izze,[Clementine],$3.39
2,1,1,Nantucket Nectar,[Apple],$3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,$2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98
5,3,1,Chicken Bowl,"[Fresh Tomato Salsa (Mild), [Rice, Cheese, Sou...",$10.98
6,3,1,Side of Chips,NaN,$1.69
7,4,1,Steak Burrito,"[Tomatillo Red Chili Salsa, [Fajita Vegetables...",$11.75
8,4,1,Steak Soft Tacos,"[Tomatillo Green Chili Salsa, [Pinto Beans, Ch...",$9.25
9,5,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Pinto...",$9.25


In [4]:
chipo.dtypes

order_id               int64
quantity               int64
item_name             object
choice_description    object
item_price            object
dtype: object

# 1. On va convertir la colonne des prix en flottants
1. crée une liste contenant toutes les valeurs en flottant
2. remplace la colonne item_price par cette liste

In [5]:
# Enlever le dollar (value[0]) de chaque chaine de caractères et convertir en flottants
prices = [float(value[1 : -1]) for value in chipo.item_price]

# remplacer la colonne
chipo.item_price = prices

In [6]:
chipo.head(5)

,order_id,quantity,item_name,choice_description,item_price
0,1,1,Chips and Fresh Tomato Salsa,NaN,2.39
1,1,1,Izze,[Clementine],3.39
2,1,1,Nantucket Nectar,[Apple],3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",16.98


# Récupère quelques stats descriptives

In [7]:
chipo.describe()

,order_id,quantity,item_price
count,4622.000000,4622.000000,4622.000000
mean,927.254868,1.075725,7.464336
std,528.890796,0.410186,4.245557
min,1.000000,1.000000,1.090000
25%,477.250000,1.000000,3.390000
50%,926.000000,1.000000,8.750000
75%,1393.000000,1.000000,9.250000
max,1834.000000,15.000000,44.250000


# Compte le nombre d'objets à la carte

utilise .nunique

In [8]:
chipo.item_name.nunique()

50

# Calcule le prix de chaque commande (order en anglais) avec groupby et stocke la série dans une variable appelée commandes

In [9]:
commandes = chipo.groupby('order_id')['item_price'].sum()
commandes.head(5)

order_id
1    11.56
2    16.98
3    12.67
4    21.00
5    13.70
Name: item_price, dtype: float64

# Quelles sont les 5 commandes les plus chères ? 
Trie la série de manière décroissante avec sort_values en passant l'argument ascending = False

In [10]:
commandes.sort_values(ascending = False).head(5)

order_id
926     205.25
1443    160.74
1483    139.00
691     118.25
1786    114.30
Name: item_price, dtype: float64

## On voudrait faire une table contenant une ligne pour chaque plat du menu, avec son prix à l'unité et sa description, qu'on va appeler menu

On commencer par retirer toutes les lignes qui ont fait la même commande avec drop_duplicates.\
Ensuite on divise le prix payé par la quantité commandée, et on en fait une nouvelle colonne \
On peut ensuite laisser de côté les colonnes inutiles avec drop, en donnant l'argument columns = \['order_id','quantity','item_price'\]

In [11]:
#On conserve une commande par item
menu = chipo.drop_duplicates(['item_name']).copy()

#On renormalise les prix à l'unité
prix_unite = menu['item_price'] / menu.quantity

#On replace une nouvelle colonne 
menu['unit_price']= prix_unite.values
menu = menu.drop(columns = ['order_id','quantity', 'item_price'])
menu.head(10)

,item_name,choice_description,unit_price
0,Chips and Fresh Tomato Salsa,NaN,2.39
1,Izze,[Clementine],3.39
2,Nantucket Nectar,[Apple],3.39
3,Chips and Tomatillo-Green Chili Salsa,NaN,2.39
4,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",8.49
6,Side of Chips,NaN,1.69
7,Steak Burrito,"[Tomatillo Red Chili Salsa, [Fajita Vegetables...",11.75
8,Steak Soft Tacos,"[Tomatillo Green Chili Salsa, [Pinto Beans, Ch...",9.25
10,Chips and Guacamole,NaN,4.45
11,Chicken Crispy Tacos,"[Roasted Chili Corn Salsa, [Fajita Vegetables,...",8.75


# Très beau menu ! On peut l'indexer par les noms des plats

In [12]:
menu.set_index('item_name')

,choice_description,unit_price
item_name,,
Chips and Fresh Tomato Salsa,NaN,2.39
Izze,[Clementine],3.39
Nantucket Nectar,[Apple],3.39
Chips and Tomatillo-Green Chili Salsa,NaN,2.39
Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",8.49
Side of Chips,NaN,1.69
Steak Burrito,"[Tomatillo Red Chili Salsa, [Fajita Vegetables...",11.75
Steak Soft Tacos,"[Tomatillo Green Chili Salsa, [Pinto Beans, Ch...",9.25
Chips and Guacamole,NaN,4.45


# Dernier problème : quel est le plat le plus populaire ?

In [13]:
chipo.groupby('item_name').sum().sort_values(by = 'quantity', ascending = False).head(1)

,order_id,quantity,item_price
item_name,,,
Chicken Bowl,713926,761,7342.73
